In [1]:
## Load the model
from langchain_ollama import ChatOllama

local_llm = 'llama3.2'
local_embedding_model = 'BAAI/bge-m3'

llm = ChatOllama(model=local_llm, temperature=0.0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0.0, format='json')

### Vector store and embedding model
*Using basic query to test the vectorstore and embedding model.*

---------------------------

In [2]:
from embedding.vector_store import VectorStore
from embedding.embedding_models import EmbeddingModels

embedding_model = EmbeddingModels().get_bge_embedding(model_name=local_embedding_model)
chroma_kb = VectorStore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/peer_kb', 
    embedding=embedding_model
)

# query_text = 'what is ADRD' # failed, no results
query_text = 'what is the specialty for Chinese language. ' 
similarity_search_res = VectorStore.retrieve_docs(query=query_text, vectorstore=chroma_kb, k=5)

for doc in similarity_search_res:
    print(doc.page_content)
    print(doc.metadata)

AttributeError: module 'decoders' has no attribute 'DecodeStream'

### Retrieve grading 
*Preventing hallucination and erroneous retrieval that is not relevant to the question but based on the idiosyncrasies of the embedding model or chunking.*

---------------------

In [1]:
from typing import List
from checkpoints.retrieval_grading import grade_retrieval
from langchain.schema import Document

# Grade the retrieval results
graded_docs = grade_retrieval(query_text, similarity_search_res, model="marco-o1")

# Print graded results
# for doc in graded_docs:
#     print(f"Document content: {doc}")
    
# Filter out documents with relevance_score lower than 0.5
class ReasonedDocument:
    """
    A class to store the document, relevance score, and reasoning.
    """
    
    document: Document # Retrieved document from vector store in Document type
    relevance_score: float # Relevance score by reasoning model
    reasoning: str # Reasons for the relevance score
    
    def __init__(self, document, relevance_score, reasoning):
        self.document = document
        self.relevance_score = relevance_score
        self.reasoning = reasoning
        
    def __str__(self):
        return f"Document: {self.document}\nRelevance Score: {self.relevance_score}\nReasoning: {self.reasoning}\n"

# filtered_docs = [doc for doc, grade in zip(similarity_search_res, graded_docs) if grade["relevance_score"] >= 0.5]

filtered_docs = []

for doc, grade in zip(similarity_search_res, graded_docs):
    if grade["relevance_score"] >= 0.5:
        reasoned_doc = ReasonedDocument(
            document=doc,
            relevance_score=grade["relevance_score"],
            reasoning=grade["reasoning"]
        )
        filtered_docs.append(reasoned_doc)


# Print filtered results
for doc in filtered_docs:
    print(doc)
    # print(doc.metadata)



2025-01-14 @ 01:22:31 | INFO     | utils.logger:<module>:56 - Logger initialized successfully


NameError: name 'query_text' is not defined

### Generate answer
*Using the graded documents to generate an answer.*

--------------------------------

In [1]:
from answer_generation import generate_answer

context_chunks = [doc.document for doc in filtered_docs]

# TODO: handle the case where context_chunks is empty, answer should specify that we don't have relevant information.

answer = generate_answer(query_text, context_chunks)

print(answer)

2025-01-13 @ 01:37:39 | INFO     | utils.logger:<module>:56 - Logger initialized successfully


NameError: name 'filtered_docs' is not defined